# Azure Cognitive Search에  CSV 파일 로드 (one-to-many)

이번 단계에서는 각 행이 개별적인 레코드/문서인 CSV 파일을 인덱싱하는 단계를 생성하고 실행하여 각 행을 Azure Cognitive Search에서 검색 가능하게 합니다. 
[참고문헌](https://learn.microsoft.com/en-us/azure/search/search-howto-index-one-to-many-blobs)에서 확인하실 수 있습니다. 

기본적으로 인덱서는 Blob 또는 파일의 내용을 단일 검색 문서로 처리합니다. 만약 검색에 좀 더 세분화된 표현을 원할 경우, ParsingMode 값을 설정하여 하나의 블롭 또는 파일에서 여러 검색 문서를 작성할 수도 있습니다.

In [17]:
import os
import json
import requests
from dotenv import load_dotenv
load_dotenv("credentials.env")

# credentials.env 파일안에 연결된 Blob 스토리지 컨테이너 이름
BLOB_CONTAINER_NAME = "cord19"

In [18]:
# 데이터 소스, 스킬셋, 인덱스 및 인덱서 이름 정의합니다.
datasource_name = "cogsrch-datasource-csv"
skillset_name = "cogsrch-skillset-csv"
index_name = "cogsrch-index-csv"
indexer_name = "cogsrch-indexer-csv"

In [19]:
# Payloads header를 설정합니다. 
headers = {'Content-Type': 'application/json','api-key': os.environ['AZURE_SEARCH_KEY']}
params = {'api-version': os.environ['AZURE_SEARCH_API_VERSION']}

## Data Source 생성

In [20]:
# data source 생성

datasource_payload = {
    "name": datasource_name,
    "description": "Demo files to demonstrate cognitive search capabilities of one-to-many.",
    "type": "azureblob",
    "credentials": {
        "connectionString": os.environ['BLOB_CONNECTION_STRING']
    },
    "container": {
        "name": BLOB_CONTAINER_NAME
    }
}
r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/datasources/" + datasource_name,
                 data=json.dumps(datasource_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

201
True


## 인덱스를 만들기 전에 데이터의 열 유형을 이해할 수 있도록 CSV 파일을 검사

In [21]:
# csv 파일을 디스크에 다운로드하고 panda 라이브러리를 사용하여 검사합니다. 
import pandas as pd
remote_file_path = "https://holstorage.blob.core.windows.net/cord19/Seoul_Commercial_Districts_v2.csv"


In [22]:
metadata = pd.read_csv(remote_file_path + os.environ['BLOB_SAS_TOKEN'])
print("No. of lines:",metadata.shape[0])

No. of lines: 859


In [23]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 859 entries, 0 to 858
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   uniqueid  859 non-null    object 
 1   기준분기      859 non-null    object 
 2   상권구분      859 non-null    object 
 3   상권지역      859 non-null    object 
 4   서비스업종     859 non-null    object 
 5   분기매출      859 non-null    float64
 6   분기당매출건수   859 non-null    int64  
 7   주중매출금액    859 non-null    float64
 8   주말매출금액    859 non-null    int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 60.5+ KB


In [24]:
simple_schema = ['기준분기', '상권구분', '상권지역', '서비스업종', '분기매출', '분기당매출건수',  '주중매출금액',  '주말매출금액', 'uniqueid' ]


def make_clickable(address):
    '''Make the url clickable'''
    return '<a href="{0}">{0}</a>'.format(address)

def preview(text):
    '''Show only a preview of the text data.'''
    return text[:30] + '...'


format_ = {'기준분기': preview, '상권구분': preview, '상권지역': preview, '서비스업종': preview, '분기매출': float, '분기당매출건수': int, '주중매출금액': float, '주말매출금액': int, 'uniqueid': preview}

metadata[simple_schema].head().style.format(format_)

,기준분기,상권구분,상권지역,서비스업종,분기매출,분기당매출건수,주중매출금액,주말매출금액,uniqueid
0,4분기...,발달상권...,대림3동사거리...,가구...,205619535.0,220,205619535.0,0,YH0...
1,4분기...,발달상권...,송파사거리(송파역)...,시계및귀금속...,47289011.0,156,47289011.0,0,YH1...
2,4분기...,발달상권...,상계백병원...,가구...,3509705.0,11,3509705.0,0,YH2...
3,4분기...,전통시장...,화곡본동시장...,가구...,6554404.0,10,6554404.0,0,YH3...
4,4분기...,발달상권...,강남을지병원...,가구...,125073744.0,108,125073744.0,0,YH4...


## Skillset 생성 - Text Splitter, Language Detection
각 행의 내용이 얼마나 큰지 모르기 때문에 각 컨텐츠 필드의 텍스트를 청크(페이지)로 분할하고 언어를 검출합니다. 

In [25]:
# 스킬셋 생성
skillset_payload = {
    "name": skillset_name,
    "description": "Splits Text and detect language",
    "skills":
    [
        {
            "@odata.type": "#Microsoft.Skills.Text.LanguageDetectionSkill",
            "description": "If you have multilingual content, adding a language code is useful for filtering",
            "context": "/document",
            "inputs": [
                {
                  "name": "text",
                  "source": "/document/abstract"
                }
            ],
            "outputs": [
                {
                  "name": "languageCode",
                  "targetName": "language"
                }
            ]
        },
        {
            "@odata.type": "#Microsoft.Skills.Text.SplitSkill",
            "context": "/document",
            "textSplitMode": "pages",
            "maximumPageLength": 5000, # 5000 is default
            "defaultLanguageCode": "en",
            "inputs": [
                {
                    "name": "text",
                    "source": "/document/abstract"
                },
                {
                    "name": "languageCode",
                    "source": "/document/language"
                }
            ],
            "outputs": [
                {
                    "name": "textItems",
                    "targetName": "pages"
                }
            ]
        }
    ],
    "cognitiveServices": {
        "@odata.type": "#Microsoft.Azure.Search.CognitiveServicesByKey",
        "description": os.environ['COG_SERVICES_NAME'],
        "key": os.environ['COG_SERVICES_KEY']
    }
}

r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/skillsets/" + skillset_name,
                 data=json.dumps(skillset_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

201
True


## Index 생성
Azure Cognitive Search에서는 블롭 인덱서와 파일 인덱서 모두 CSV 파일의 각 행을 별도의 검색 문서로 구분하는 구문 분석을 지원합니다. 


### **중요**:
전 단계와 현 단계에서 볼 수 있듯이 스키마는 'id, title, content, chunks, language, name, location'의 7개 필수 필드가 있습니다. 이 필드들은 데이터 소스와 관계없이 작성하는 모든 인덱스에 존재해야 하며 검색 엔진이 관련 문서를 검색할 수 있도록 추가 필드를 추가하는 것도 가능합니다. 

In [26]:
index_payload = {
    "name": index_name,  
    "fields": [
        {"name": "id", "type": "Edm.String", "key": "true", "searchable": "false", "retrievable": "true", "facetable": "false", "filterable": "false", "sortable": "false"},
        {"name": "title", "type": "Edm.String", "searchable": "true", "retrievable": "true", "facetable": "false", "filterable": "true", "sortable": "false"},
        {"name": "content", "type": "Edm.String", "searchable": "true", "retrievable": "true", "facetable": "false", "filterable": "false", "sortable": "false"},
        {"name": "content2", "type": "Edm.String", "searchable": "true", "retrievable": "true", "facetable": "false", "filterable": "false", "sortable": "false"},
        {"name": "content3", "type": "Edm.String", "searchable": "true", "retrievable": "true", "facetable": "false", "filterable": "false", "sortable": "false"},
        {"name": "content4", "type": "Edm.String", "searchable": "true", "retrievable": "true", "facetable": "false", "filterable": "false", "sortable": "false"},
        {"name": "chunks","type": "Collection(Edm.String)", "searchable": "false", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "language", "type": "Edm.String", "searchable": "false", "retrievable": "true", "sortable": "true", "filterable": "true", "facetable": "true"},
        {"name": "name", "type": "Edm.String", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "location", "type": "Edm.String", "searchable": "false", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "vectorized", "type": "Edm.Boolean", "searchable": "false", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "authors", "type": "Edm.String", "searchable": "true", "retrievable": "true", "facetable": "false", "filterable": "false", "sortable": "false"},
        {"name": "metadata_storage_name", "type": "Edm.String", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "metadata_storage_path", "type":"Edm.String", "searchable": "false", "retrievable": "true", "filterable": "false", "sortable": "false"},
        {"name": "metadata_storage_last_modified", "type":"Edm.DateTimeOffset", "searchable": "false", "retrievable": "false", "filterable": "false", "sortable": "false"}
    ],
    "semantic": {
        "configurations": [
            {
                "name": "my-semantic-config",
                "prioritizedFields": {
                    "titleField": 
                        {
                            "fieldName": "title"
                        },
                    "prioritizedContentFields": [
                        { 
                            "fieldName":"content" 
                        }
                    ]
                }
            }
        ]
    }
}

r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + index_name,
                 data=json.dumps(index_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

201
True


## Indexer 생성 및 실행 - (runs the pipeline)
CSV 데이터를 사용하여 일대다 인덱서를 만들려면 텍스트 구문 분석 모드를 사용하여 인덱서의 정의를 만들거나 업데이트 해야합니다

In [27]:
# format_ = {'기준분기': preview, '상권구분': preview, '상권지역': preview, '서비스업종': preview, '분기매출': float, '분기당매출건수': int, '주중매출금액': float, '주말매출금액': int, 'uniqueid': preview}

In [28]:
indexer_payload = {
    "name": indexer_name,
    "dataSourceName": datasource_name,
    "targetIndexName": index_name,
    "skillsetName": skillset_name,
    "schedule" : { "interval" : "PT2H"},
    "fieldMappings": [
        {
          "sourceFieldName" : "uniqueid",
          "targetFieldName" : "id",
        },
        {
          "sourceFieldName" : "기준분기",
          "targetFieldName" : "content"
        },
        {
          "sourceFieldName" : "분기매출",
          "targetFieldName" : "content2"
        },
        {
          "sourceFieldName" : "주중매출금액",
          "targetFieldName" : "content3"
        },
        {
          "sourceFieldName" : "주말매출금액",
          "targetFieldName" : "content4"
        },
        {
          "sourceFieldName" : "서비스업종",
          "targetFieldName" : "name"
        },
        {
          "sourceFieldName" : "상권지역",
          "targetFieldName" : "location"
        }
    ],
    "outputFieldMappings":
    [
        {
            "sourceFieldName": "/document/language",
            "targetFieldName": "language"
        },
        {
            "sourceFieldName": "/document/pages/*",
            "targetFieldName": "chunks"
        }
    ],
    "parameters" : { 
        "configuration" : { 
            "dataToExtract": "contentAndMetadata",
            "parsingMode" : "delimitedText", 
            "firstLineContainsHeaders" : True,
            "delimitedTextDelimiter": ","
        } 
    }
}
r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexers/" + indexer_name,
                 data=json.dumps(indexer_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

201
True


In [30]:
# 인덱서 상태를 가져와 실행 중임을 확인합니다
r = requests.get(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexers/" + indexer_name +
                 "/status", headers=headers, params=params)
# pprint(json.dumps(r.json(), indent=1))
print(r.status_code)
print("Status:",r.json().get('lastResult').get('status'))
print("Items Processed:",r.json().get('lastResult').get('itemsProcessed'))
print(r.ok)

200
Status: inProgress
Items Processed: 0
True


In [31]:
print(r.text)

{"@odata.context":"https://cog-search-iwdkwba7nutto.search.windows.net/$metadata#Microsoft.Azure.Search.V2023_07_01_Preview.IndexerExecutionInfo","name":"cogsrch-indexer-csv","status":"running","lastResult":{"status":"inProgress","statusDetail":null,"errorMessage":null,"startTime":"2023-10-20T05:44:40.935Z","endTime":null,"itemsProcessed":0,"itemsFailed":0,"initialTrackingState":null,"finalTrackingState":null,"mode":"indexingAllDocs","errors":[],"warnings":[],"metrics":null},"executionHistory":[],"limits":{"maxRunTime":"PT2H","maxDocumentExtractionSize":134217728,"maxDocumentContentCharactersToExtract":4194304},"currentState":{"mode":"indexingAllDocs","allDocsInitialTrackingState":null,"allDocsFinalTrackingState":null,"resetDocsInitialTrackingState":null,"resetDocsFinalTrackingState":null,"resetDocumentKeys":[],"resetDatasourceDocumentIds":[]}}


## 벡터 기반 인덱스 생성

In [32]:
index_payload = {
    "name": index_name + "-vector",
    "fields": [
        {"name": "id", "type": "Edm.String", "key": "true", "filterable": "true" },
        {"name": "title","type": "Edm.String","searchable": "true","retrievable": "true"},
        {"name": "chunk","type": "Edm.String","searchable": "true","retrievable": "true"},
        {"name": "chunkVector","type": "Collection(Edm.Single)","searchable": "true","retrievable": "true","dimensions": 1536,"vectorSearchConfiguration": "vectorConfig"},
        {"name": "name", "type": "Edm.String", "searchable": "true", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},
        {"name": "location", "type": "Edm.String", "searchable": "false", "retrievable": "true", "sortable": "false", "filterable": "false", "facetable": "false"},

    ],
    "vectorSearch": {
        "algorithmConfigurations": [
            {
                "name": "vectorConfig",
                "kind": "hnsw"
            }
        ]
    },
    "semantic": {
        "configurations": [
            {
                "name": "my-semantic-config",
                "prioritizedFields": {
                    "titleField": {
                        "fieldName": "title"
                    },
                    "prioritizedContentFields": [
                        {
                            "fieldName": "chunk"
                        }
                    ],
                    "prioritizedKeywordsFields": []
                }
            }
        ]
    }
}

r = requests.put(os.environ['AZURE_SEARCH_ENDPOINT'] + "/indexes/" + index_name + "-vector",
                 data=json.dumps(index_payload), headers=headers, params=params)
print(r.status_code)
print(r.ok)

201
True


# Reference

- https://learn.microsoft.com/en-us/azure/search/search-howto-index-csv-blobs
- https://learn.microsoft.com/en-us/azure/search/knowledge-store-create-rest

